## 下载数据集

In [1]:
from modelscope.msdatasets import MsDataset
import json
import random

random.seed(42)

ds = MsDataset.load('krisfu/delicate_medical_r1_data', subset_name='default', split='train')
data_list = list(ds)
random.shuffle(data_list)

split_idx = int(len(data_list) * 0.9)

train_data = data_list[:split_idx]
val_data = data_list[split_idx:]

with open('../dataset/delicate_medical_r1/train.jsonl', 'w', encoding='utf-8') as f:
    for item in train_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

with open('../dataset/delicate_medical_r1/val.jsonl', 'w', encoding='utf-8') as f:
    for item in val_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"The dataset has been split successfully.")
print(f"Train Set Size：{len(train_data)}")
print(f"Val Set Size：{len(val_data)}")

/home/fine/uv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The dataset has been split successfully.
Train Set Size：2166
Val Set Size：241


## 加载模型和分词器

In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM

/home/fine/uv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/data/download-model/Qwen3-0.6B"

In [3]:
# Transformers加载模型权重
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,use_cache=False)

## 调整数据的格式用于训练

In [4]:
import os
datset_path='../dataset/delicate_medical_r1/'
train_dataset_path = os.path.join(datset_path,"train.jsonl")
test_dataset_path = os.path.join(datset_path,"val.jsonl")

In [5]:
from datasets import load_dataset

In [6]:
dataset=load_dataset('json',data_files={
    'train':train_dataset_path,
    'validation':test_dataset_path
})

In [7]:
def process_func(example):
    """
    将数据集进行预处理
    """ 
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n{PROMPT}<|im_end|>\n<|im_start|>user\n{example['question']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False
    )
    ## tokenizer.pad_token_id='<|endoftext|>'
    response = tokenizer(
        f"<think>{example['think']}</think>\n{example['answer']}", 
        add_special_tokens=False
    )
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = (
        instruction["attention_mask"] + response["attention_mask"] + [1]
    )
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    ## 转成tensor
    # labels=torch.tensor(labels).to("cuda:0")
    # input_ids=torch.tensor(input_ids).to("cuda:0")
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}   


In [8]:
PROMPT = "你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。"
MAX_LENGTH = 2048

In [9]:
dataset=dataset.map(process_func)

Map: 100%|██████████| 241/241 [00:00<00:00, 678.84 examples/s]


## 准备lora

In [11]:
from peft import LoraConfig, TaskType, get_peft_model

In [12]:
# 配置lora
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

model = get_peft_model(model, config)

## 准备swanlab

In [10]:
import swanlab

In [11]:
swanlab.config.update({
    "model": "Qwen3-0.6B",
    "prompt": PROMPT,
    "data_max_length": MAX_LENGTH,
    })

## 准备训练

In [12]:
from transformers import TrainingArguments,Trainer,DataCollatorForSeq2Seq
import torch

In [13]:
args = TrainingArguments(
    output_dir="../output/Qwen3-0.6B",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=10,
    num_train_epochs=2,
    save_steps=400,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="swanlab",
    run_name="qwen3-0.6B",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## 开始训练

In [14]:
trainer.train()

swanlab: Tracking run with swanlab version 0.6.4                                   
swanlab: Run data will be saved locally in /home/adminad/zhangdw/workspace/finetuning/transformers/swanlog/run-20250628_164547-a3b1799d
swanlab: 👋 Hi zhangdw156, welcome to swanlab!
swanlab: Syncing run qwen3-0.6B to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@zhangdw156/transformers
swanlab: 🚀 View run at https://swanlab.cn/@zhangdw156/transformers/runs/lyic3titezexqrdlvanps


Step,Training Loss,Validation Loss
100,0.213800,1.731899


/home/fine/uv/transformers/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=136, training_loss=0.2293119903872995, metrics={'train_runtime': 321.7503, 'train_samples_per_second': 13.464, 'train_steps_per_second': 0.423, 'total_flos': 1.2680557687209984e+16, 'train_loss': 0.2293119903872995, 'epoch': 2.0})

## 验证结果

In [33]:
len(dataset['validation'])

241

In [19]:
test_text_list = []

In [21]:
messages=[
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{dataset['validation']['question'][0]}"}
]

In [22]:
messages

[{'role': 'system', 'content': '你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。'},
 {'role': 'user',
  'content': '1895年德国物理学教授伦琴的发现对医学影像学的发展有何具体影响？请从技术进步、学科建立和临床应用三个方面进行分析。'}]

In [23]:
text=tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

In [24]:
text

'<|im_start|>system\n你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。<|im_end|>\n<|im_start|>user\n1895年德国物理学教授伦琴的发现对医学影像学的发展有何具体影响？请从技术进步、学科建立和临床应用三个方面进行分析。<|im_end|>\n<|im_start|>assistant\n'

In [26]:
model_inputs = tokenizer([text], return_tensors="pt").to('cuda:0')

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=MAX_LENGTH,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/fine/uv/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [27]:
response

'<think>嗯，用户问的是1895年德国物理学家伦琴的发现对医学影像学的影响，分技术、学科和临床应用三个点。首先，我需要回忆一下伦琴的基本知识。记得他是一位年轻的科学家，可能对医学影像学不太熟悉，但后来因为他的研究而变得很有名。\n\n技术进步方面，我记得他可能改进了X光技术，比如X线的剂量控制，或者让X线更安全，这样医生能更放心使用。另外，他可能推动了影像学的临床应用，比如CT或MRI的前身。不过具体细节可能需要查证，比如是否提到X线的高分辨率或对比度提升，或者是否影响了诊断的准确性。比如，如果X线的对比度更好，可能让诊断更准确，减少误诊。\n\n接下来是学科建立。可能他推动了影像学的学科，比如影像学诊断学，或者建立了新的研究领域，比如神经影像学。比如，他可能研究神经影像学，或者影像学中的神经影像学，这可能属于神经影像学的前身。另外，可能影响了诊断学，比如影像学诊断学的形成，或者让影像学的诊断学更具体，比如影像学诊断学的各个门类，比如内镜学、CT学等。\n\n临床应用方面，可能他推动了影像学的应用，比如CT和MRI的前身。比如，早期的CT和MRI技术可能基于他的研究，或者后来的影像学技术更进一步。比如，现在使用的CT系统可能更精确，或者MRI的高分辨率MRI技术可能来自他的研究。此外，他可能促进了影像学的临床实践，比如让医生更放心使用X线，减少辐射暴露，从而改善诊断的安全性和准确性。\n\n需要确认这些点是否准确。比如，是否提到X线的剂量控制？或者是否影响了诊断的准确性？比如，如果X线的对比度更好，诊断更准确，减少误诊。另外，是否影响了诊断学的结构，比如影像学诊断学的形成，或者让影像学的诊断学更具体？\n\n可能还要考虑他的研究对医学影像学的推动作用，比如影像学诊断学的学科建设，或者影像学诊断学的各个门类，比如内镜学、CT学等，这些可能都是基于他的研究或进一步发展。\n\n总结下来，技术进步可能涉及X线的高分辨率和对比度，学科建立涉及神经影像学，临床应用涉及CT和MRI的前身，以及诊断学的形成。需要把这些点组织成连贯的思考过程，确保逻辑清晰，覆盖所有方面。\n</think>\n1895年，德国物理学家伦琴的发现对医学影像学的发展产生了重要影响。首先，他通过改进X线技术，使得X线能够更安全地用于临床诊断，减少了对患者辐射剂量的担忧，从而提高了诊断的准确性。其次

In [31]:
response_text = f"""
Question: {{dataset['validation']['question'][0]}}
LLM:{response}
"""

In [32]:
response_text

"\nQuestion: {dataset['validation']['question'][0]}\nLLM:<think>嗯，用户问的是1895年德国物理学家伦琴的发现对医学影像学的影响，分技术、学科和临床应用三个点。首先，我需要回忆一下伦琴的基本知识。记得他是一位年轻的科学家，可能对医学影像学不太熟悉，但后来因为他的研究而变得很有名。\n\n技术进步方面，我记得他可能改进了X光技术，比如X线的剂量控制，或者让X线更安全，这样医生能更放心使用。另外，他可能推动了影像学的临床应用，比如CT或MRI的前身。不过具体细节可能需要查证，比如是否提到X线的高分辨率或对比度提升，或者是否影响了诊断的准确性。比如，如果X线的对比度更好，可能让诊断更准确，减少误诊。\n\n接下来是学科建立。可能他推动了影像学的学科，比如影像学诊断学，或者建立了新的研究领域，比如神经影像学。比如，他可能研究神经影像学，或者影像学中的神经影像学，这可能属于神经影像学的前身。另外，可能影响了诊断学，比如影像学诊断学的形成，或者让影像学的诊断学更具体，比如影像学诊断学的各个门类，比如内镜学、CT学等。\n\n临床应用方面，可能他推动了影像学的应用，比如CT和MRI的前身。比如，早期的CT和MRI技术可能基于他的研究，或者后来的影像学技术更进一步。比如，现在使用的CT系统可能更精确，或者MRI的高分辨率MRI技术可能来自他的研究。此外，他可能促进了影像学的临床实践，比如让医生更放心使用X线，减少辐射暴露，从而改善诊断的安全性和准确性。\n\n需要确认这些点是否准确。比如，是否提到X线的剂量控制？或者是否影响了诊断的准确性？比如，如果X线的对比度更好，诊断更准确，减少误诊。另外，是否影响了诊断学的结构，比如影像学诊断学的形成，或者让影像学的诊断学更具体？\n\n可能还要考虑他的研究对医学影像学的推动作用，比如影像学诊断学的学科建设，或者影像学诊断学的各个门类，比如内镜学、CT学等，这些可能都是基于他的研究或进一步发展。\n\n总结下来，技术进步可能涉及X线的高分辨率和对比度，学科建立涉及神经影像学，临床应用涉及CT和MRI的前身，以及诊断学的形成。需要把这些点组织成连贯的思考过程，确保逻辑清晰，覆盖所有方面。\n</think>\n1895年，德国物理学家伦琴的发现对医学影像学的发展产生了重要影响。首先

In [39]:
import random
test_text_list = []
for i in random.sample(range(len(dataset['validation'])),3):
    messages=[
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{dataset['validation']['question'][0]}"}
    ]
    text=tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to('cuda:0')

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=MAX_LENGTH,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    response_text = f"""Question: {dataset['validation']['question'][i]}\nLLM:{response}"""

    test_text_list.append(swanlab.Text(response_text))
    

In [40]:
swanlab.log({"Prediction": test_text_list})

{'Prediction': ['Question: 医生，我最近查了一些资料，发现库欣综合征好像有几种不同的类型，能具体给我讲讲每种类型的临床表现和实验室检查的特点吗？\nLLM:<think>嗯，用户问的是1895年德国物理学家伦琴的发现对医学影像学的具体影响，特别是技术进步、学科建立和临床应用三个层面。首先，我需要回忆一下伦琴的基本知识。记得他之前学过医学，可能对影像学有一定的背景。用户可能想知道这些影响的具体表现，比如技术上的具体进步，比如CT或MRI的出现，或者影像学技术的改进。\n\n首先，技术进步方面，可能包括CT和MRI的出现。比如，CT是计算机断层，MRI是磁共振成像，它们极大地提高了影像的清晰度和对比度，让医生可以看到更细节的病变。不过用户可能不太了解这些技术的具体细节，所以需要解释清楚。另外，可能还有其他技术，比如超声技术，或者更早的影像学设备，比如X光或CT，但用户的问题主要集中在技术上，所以重点放在这些技术。\n\n然后是学科建立。医学影像学需要将不同的技术结合起来，比如CT和MRI的结合，可能形成CT-MR技术。比如，CT-MR结合了计算机断层和磁共振，这应该是医学影像学领域的一个重要突破。此外，可能还有其他学科，比如放射学、内镜学等，这些学科后来也发展，但用户提到的三个层面可能覆盖了主要学科。\n\n接下来是临床应用。临床医生如何利用这些技术。比如，CT能帮助诊断肿瘤，MRI可以评估脑部病变，比如肿瘤的大小、形状，MRI能帮助医生理解病变的性质，比如恶性肿瘤是否侵犯周围组织。此外，可能还有CT在手术中的应用，比如评估肿瘤是否侵犯周围结构，MRI在诊断早期发现肿瘤，或者在CT上评估肿瘤的代谢情况。\n\n需要确认是否有其他技术，比如超声技术，或者是否还有其他影像学技术被提及。但根据答案，重点在技术进步、学科建立和临床应用，所以可能不需要扩展太多。另外，要确保回答结构清晰，分三个层面，每个层面详细说明影响的具体方面。\n\n可能需要检查是否有遗漏的点，比如是否提到技术的具体细节，比如超声的原理，或者是否提到MR技术。不过根据答案，可能不需要太深入，保持简洁。还要注意术语的准确性，比如MRI是磁共振成像，CT是计算机断层，MRI是磁共振成像，所以要正确使用缩写。\n\n总结下来，回答需要先总述技术进步，再分学科建立，最后讨论临床应用的具体例子

In [41]:
swanlab.finish()

swanlab: 🏠 View project at https://swanlab.cn/@zhangdw156/transformers
swanlab: 🚀 View run at https://swanlab.cn/@zhangdw156/transformers/runs/lyic3titezexqrdlvanps
swanlab: Waiting for uploading complete
                                                                                                    